In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pickle
import ipdb
import os
from glob import glob
import pandas as pd
import cv2
import skimage.io
import skimage.transform
from PIL import ImageFile
tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


# Data Loader and Random Square Crop

The data loader function defined as load_image, reads the image from the given path and reshapes it to the required height and width given by the use.

The function crop_random helps us crop random square from the input image with the defined width and height and overlap. We iterate with changing the position and the given maximum overlap criteria for the training step. The test set is also cropped using the same function to obtain the corrupted images for evaluation.


In [2]:
#def load_image( path, height=128, width=128 ):
def load_image( path, pre_height=146, pre_width=146, height=128, width=128 ):

    try:
        img = skimage.io.imread( path ).astype( float )
    except:
        return None

    img /= 255.

    if img is None: return None
    if len(img.shape) < 2: return None
    if len(img.shape) == 4: return None
    if len(img.shape) == 2: img=np.tile(img[:,:,None], 3)
    if img.shape[2] == 4: img=img[:,:,:3]
    if img.shape[2] > 4: return None

    short_edge = min( img.shape[:2] )
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy:yy+short_edge, xx:xx+short_edge]
    resized_img = skimage.transform.resize( crop_img, [pre_height,pre_width] )

    rand_y = np.random.randint(0, pre_height - height)
    rand_x = np.random.randint(0, pre_width - width)

    resized_img = resized_img[ rand_y:rand_y+height, rand_x:rand_x+width, : ]

    return (resized_img * 2)-1 #(resized_img - 127.5)/127.5

def crop_random(image_ori, width=64,height=64, x=None, y=None, overlap=7):

    if image_ori is None: return None
    random_y = np.random.randint(overlap,height-overlap) if x is None else x
    random_x = np.random.randint(overlap,width-overlap) if y is None else y

    image = image_ori.copy()
    crop = image_ori.copy()
    crop = crop[random_y:random_y+height, random_x:random_x+width]
    image[random_y + overlap:random_y+height - overlap, random_x + overlap:random_x+width - overlap, 0] = 2*117. / 255. - 1.
    image[random_y + overlap:random_y+height - overlap, random_x + overlap:random_x+width - overlap, 1] = 2*104. / 255. - 1.
    image[random_y + overlap:random_y+height - overlap, random_x + overlap:random_x+width - overlap, 2] = 2*123. / 255. - 1.
    return image, crop, random_x, random_y


# Model:
The model is defined as follows in the class Model() using tensorflow. Four functions for creating new conv layer, deconv layer, fully connected layer and channel wise fully connected layer are defined. The reconstruction function defines the bottle neck architecture of encoder and decoder with channel wise fully connected layer in between. And Adversial part is created accordingly

In [3]:
class Model():
    def __init__(self):
        pass

    def new_conv_layer( self, bottom, filter_shape, activation=tf.identity, padding='SAME', stride=1, name=None ):
        with tf.compat.v1.variable_scope( name ):
            w = tf.get_variable(
                    "W",
                    shape=filter_shape,
                    initializer=tf.random_normal_initializer(0., 0.005))
            b = tf.get_variable(
                    "b",
                    shape=filter_shape[-1],
                    initializer=tf.constant_initializer(0.))

            conv = tf.nn.conv2d( bottom, w, [1,stride,stride,1], padding=padding)
            bias = activation(tf.nn.bias_add(conv, b))

        return bias #relu

    def new_deconv_layer(self, bottom, filter_shape, output_shape, activation=tf.identity, padding='SAME', stride=1, name=None):
        with tf.compat.v1.variable_scope(name):
            W = tf.get_variable(
                    "W",
                    shape=filter_shape,
                    initializer=tf.random_normal_initializer(0., 0.005))
            b = tf.get_variable(
                    "b",
                    shape=filter_shape[-2],
                    initializer=tf.constant_initializer(0.))
            deconv = tf.nn.conv2d_transpose( bottom, W, output_shape, [1,stride,stride,1], padding=padding)
            bias = activation(tf.nn.bias_add(deconv, b))

        return bias

    def new_fc_layer( self, bottom, output_size, name ):
        shape = bottom.get_shape().as_list()
        dim = np.prod( shape[1:] )
        x = tf.reshape( bottom, [-1, dim])
        input_size = dim

        with tf.compat.v1.variable_scope(name):
            w = tf.get_variable(
                    "W",
                    shape=[input_size, output_size],
                    initializer=tf.random_normal_initializer(0., 0.005))
            b = tf.get_variable(
                    "b",
                    shape=[output_size],
                    initializer=tf.constant_initializer(0.))
            fc = tf.nn.bias_add( tf.matmul(x, w), b)

        return fc

    def channel_wise_fc_layer(self, input, name): # bottom: (7x7x512)
        _, width, height, n_feat_map = input.get_shape().as_list()
        input_reshape = tf.reshape( input, [-1, width*height, n_feat_map] )
        input_transpose = tf.transpose( input_reshape, [2,0,1] )

        with tf.compat.v1.variable_scope(name):
            W = tf.get_variable(
                    "W",
                    shape=[n_feat_map,width*height, width*height], # (512,49,49)
                    initializer=tf.random_normal_initializer(0., 0.005))
            output = tf.batch_matmul(input_transpose, W)

        output_transpose = tf.transpose(output, [1,2,0])
        output_reshape = tf.reshape( output_transpose, [-1, height, width, n_feat_map] )

        return output_reshape

    def leaky_relu(self, bottom, leak=0.1):
        return tf.maximum(leak*bottom, bottom)

    def batchnorm(self, bottom, is_train, epsilon=1e-8, name=None):
        bottom = tf.clip_by_value( bottom, -100., 100.)
        depth = bottom.get_shape().as_list()[-1]

        with tf.compat.v1.variable_scope(name):

            gamma = tf.get_variable("gamma", [depth], initializer=tf.constant_initializer(1.))
            beta  = tf.get_variable("beta" , [depth], initializer=tf.constant_initializer(0.))

            batch_mean, batch_var = tf.nn.moments(bottom, [0,1,2], name='moments')
            ema = tf.train.ExponentialMovingAverage(decay=0.5)


            def update():
                with tf.control_dependencies([ema_apply_op]):
                    return tf.identity(batch_mean), tf.identity(batch_var)

            ema_apply_op = ema.apply([batch_mean, batch_var])
            ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
            mean, var = tf.cond(
                    is_train,
                    update,
                    lambda: (ema_mean, ema_var) )

            normed = tf.nn.batch_norm_with_global_normalization(bottom, mean, var, beta, gamma, epsilon, False)
        return normed

    def build_reconstruction( self, images, is_train ):
        batch_size = images.get_shape().as_list()[0]

        with tf.compat.v1.variable_scope('GEN'):
            conv1 = self.new_conv_layer(images, [4,4,3,64], stride=2, name="conv1" )
            bn1 = self.leaky_relu(self.batchnorm(conv1, is_train, name='bn1'))
            conv2 = self.new_conv_layer(bn1, [4,4,64,64], stride=2, name="conv2" )
            bn2 = self.leaky_relu(self.batchnorm(conv2, is_train, name='bn2'))
            conv3 = self.new_conv_layer(bn2, [4,4,64,128], stride=2, name="conv3")
            bn3 = self.leaky_relu(self.batchnorm(conv3, is_train, name='bn3'))
            conv4 = self.new_conv_layer(bn3, [4,4,128,256], stride=2, name="conv4")
            bn4 = self.leaky_relu(self.batchnorm(conv4, is_train, name='bn4'))
            conv5 = self.new_conv_layer(bn4, [4,4,256,512], stride=2, name="conv5")
            bn5 = self.leaky_relu(self.batchnorm(conv5, is_train, name='bn5'))
            conv6 = self.new_conv_layer(bn5, [4,4,512,4000], stride=2, padding='VALID', name='conv6')
            bn6 = self.leaky_relu(self.batchnorm(conv6, is_train, name='bn6'))

            deconv4 = self.new_deconv_layer( bn6, [4,4,512,4000], conv5.get_shape().as_list(), padding='VALID', stride=2, name="deconv4")
            debn4 = tf.nn.relu(self.batchnorm(deconv4, is_train, name='debn4'))
            deconv3 = self.new_deconv_layer( debn4, [4,4,256,512], conv4.get_shape().as_list(), stride=2, name="deconv3")
            debn3 = tf.nn.relu(self.batchnorm(deconv3, is_train, name='debn3'))
            deconv2 = self.new_deconv_layer( debn3, [4,4,128,256], conv3.get_shape().as_list(), stride=2, name="deconv2")
            debn2 = tf.nn.relu(self.batchnorm(deconv2, is_train, name='debn2'))
            deconv1 = self.new_deconv_layer( debn2, [4,4,64,128], conv2.get_shape().as_list(), stride=2, name="deconv1")
            debn1 = tf.nn.relu(self.batchnorm(deconv1, is_train, name='debn1'))
            recon = self.new_deconv_layer( debn1, [4,4,3,64], [batch_size,64,64,3], stride=2, name="recon")

        return bn1, bn2, bn3, bn4, bn5, bn6, debn4, debn3, debn2, debn1, recon, tf.nn.tanh(recon)

    def build_adversarial(self, images, is_train, reuse=None):
        with tf.compat.v1.variable_scope('DIS', reuse=tf.AUTO_REUSE):
            conv1 = self.new_conv_layer(images, [4,4,3,64], stride=2, name="conv1" )
            bn1 = self.leaky_relu(self.batchnorm(conv1, is_train, name='bn1'))
            conv2 = self.new_conv_layer(bn1, [4,4,64,128], stride=2, name="conv2")
            bn2 = self.leaky_relu(self.batchnorm(conv2, is_train, name='bn2'))
            conv3 = self.new_conv_layer(bn2, [4,4,128,256], stride=2, name="conv3")
            bn3 = self.leaky_relu(self.batchnorm(conv3, is_train, name='bn3'))
            conv4 = self.new_conv_layer(bn3, [4,4,256,512], stride=2, name="conv4")
            bn4 = self.leaky_relu(self.batchnorm(conv4, is_train, name='bn4'))

            output = self.new_fc_layer( bn4, output_size=1, name='output')

        return output[:,0]

# Training Paris-Streetview dataset
We use the following hyperparameters as discussed in the paper:
- epochs = 30
- learning_rate_ = 0.002
- weight_decay_rate =  0.00001
- momentum = 0.9
- batch_size = 100
- lambda_recon = 0.999
- lambda_adv = 0.001
- overlap_size = 7
- hiding_size = 64

The dataset used is Paris street view dataset which has also been tested in the paper. The dataset consists of 
- Training images =14,900 images
- Test images= 100

In [ ]:
tf.reset_default_graph() 
n_epochs = 30
learning_rate_val = 0.002
weight_decay_rate =  0.00001
momentum = 0.9
batch_size = 100
lambda_recon = 0.999
lambda_adv = 0.001

overlap_size = 7
hiding_size = 64

trainset_path = '../data/paris_trainset.pickle'
testset_path  = '../data/paris_testset.pickle'
dataset_path = 'C:/Users/91709/Desktop/CV/Paris_street_view'
model_path = 'C:/Users/91709/Desktop/CV/models/Paris2/'
result_path= '../results/Paris2/'
pretrained_model_path = 'C:/Users/91709/Desktop/CV/models/Paris2/model-24'

if not os.path.exists(model_path):
    os.makedirs( model_path )

if not os.path.exists(result_path):
    os.makedirs( result_path )

if not os.path.exists( trainset_path ) or not os.path.exists( testset_path ):

    trainset_dir = os.path.join( dataset_path, 'paris_train_original' )
    testset_dir = os.path.join( dataset_path, 'paris_eval_gt' )

    trainset = pd.DataFrame({'image_path': map(lambda x: os.path.join( trainset_dir, x ), os.listdir(trainset_dir))})
    testset = pd.DataFrame({'image_path': map(lambda x: os.path.join( testset_dir, x ), os.listdir(testset_dir))})

    trainset.to_pickle( trainset_path )
    testset.to_pickle( testset_path )
else:
    trainset = pd.read_pickle( trainset_path )
    testset = pd.read_pickle( testset_path )

testset.index = range(len(testset))
testset = testset.loc[np.random.permutation(len(testset))]
is_train = tf.placeholder( tf.bool )

learning_rate = tf.placeholder( tf.float32, [])
images_tf = tf.placeholder( tf.float32, [batch_size, 128, 128, 3], name="images")

labels_D = tf.concat([tf.ones([batch_size]), tf.zeros([batch_size])],0)
labels_G = tf.ones([batch_size])
images_hiding = tf.placeholder( tf.float32, [batch_size, hiding_size, hiding_size, 3], name='images_hiding')

model = Model()

bn1, bn2, bn3, bn4, bn5, bn6, debn4, debn3, debn2, debn1, reconstruction_ori, reconstruction = model.build_reconstruction(images_tf, is_train)
adversarial_pos = model.build_adversarial(images_hiding, is_train)
adversarial_neg = model.build_adversarial(reconstruction, is_train, reuse=True)
adversarial_all = tf.concat([adversarial_pos, adversarial_neg],0)

# Applying bigger loss for overlapping region
mask_recon = tf.pad(tf.ones([hiding_size - 2*overlap_size, hiding_size - 2*overlap_size]), [[overlap_size,overlap_size], [overlap_size,overlap_size]])
mask_recon = tf.reshape(mask_recon, [hiding_size, hiding_size, 1])
mask_recon = tf.concat([mask_recon]*3,2)
mask_overlap = 1 - mask_recon

loss_recon_ori = tf.square( images_hiding - reconstruction )
loss_recon_center = tf.reduce_mean(tf.sqrt( 1e-5 + tf.reduce_sum(loss_recon_ori * mask_recon, [1,2,3])))  # Loss for non-overlapping region
loss_recon_overlap = tf.reduce_mean(tf.sqrt( 1e-5 + tf.reduce_sum(loss_recon_ori * mask_overlap, [1,2,3]))) * 10. # Loss for overlapping region
loss_recon = loss_recon_center + loss_recon_overlap

loss_adv_D = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(logits=adversarial_all, labels=labels_D))
loss_adv_G = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(logits=adversarial_neg, labels=labels_G))

loss_G = loss_adv_G * lambda_adv + loss_recon * lambda_recon
loss_D = loss_adv_D # * lambda_adv

var_G = list(filter( lambda x: x.name.startswith('GEN'), tf.trainable_variables()))
var_D = list(filter( lambda x: x.name.startswith('DIS'), tf.trainable_variables()))

W_G = filter(lambda x: x.name.endswith('W:0'), var_G)
W_D = filter(lambda x: x.name.endswith('W:0'), var_D)

loss_G += weight_decay_rate * tf.reduce_mean(tf.stack( list(map(lambda x: tf.nn.l2_loss(x), W_G))))
loss_D += weight_decay_rate * tf.reduce_mean(tf.stack( list(map(lambda x: tf.nn.l2_loss(x), W_D))))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

optimizer_G = tf.train.AdamOptimizer( learning_rate=learning_rate, beta1=0.5 )
grads_vars_G = optimizer_G.compute_gradients( loss_G, var_list=var_G )
#grads_vars_G = map(lambda gv: [tf.clip_by_value(gv[0], -10., 10.), gv[1]], grads_vars_G)
train_op_G = optimizer_G.apply_gradients( grads_vars_G )

optimizer_D = tf.train.AdamOptimizer( learning_rate=learning_rate, beta1=0.5 )
grads_vars_D = optimizer_D.compute_gradients( loss_D, var_list=var_D )
#grads_vars_D = map(lambda gv: [tf.clip_by_value(gv[0], -10., 10.), gv[1]], grads_vars_D)
train_op_D = optimizer_D.apply_gradients( grads_vars_D )

saver = tf.train.Saver(max_to_keep=100)

tf.initialize_all_variables().run()

if pretrained_model_path is not None and os.path.exists( pretrained_model_path ):
    saver.restore( sess, pretrained_model_path )

iters = 0

loss_D_val = 0.
loss_G_val = 0.

for epoch in range(n_epochs):
    trainset.index = range(len(trainset))
    trainset = trainset.loc[np.random.permutation(len(trainset))]

    for start,end in zip(
            range(0, len(trainset), batch_size),
            range(batch_size, len(trainset), batch_size)):

        image_paths = trainset[start:end]['image_path'].values
        images_ori = list(map(lambda x: load_image( x ), image_paths))

        if iters % 2 == 0:
            images_ori = list(map(lambda img: img[:,::-1,:], images_ori))

        is_none = np.sum(list(map(lambda x: x is None, images_ori)))
        if is_none > 0: continue

        images_crops = list(map(lambda x: crop_random(x, x=32, y=32), images_ori))
        images, crops,_,_ = zip(*images_crops)

        # Printing activations every 100 iterations
        if iters % 100 == 0:
            test_image_paths = testset[:batch_size]['image_path'].values
            test_images_ori = map(lambda x: load_image(x), test_image_paths)

            test_images_crop = map(lambda x: crop_random(x, x=32, y=32), test_images_ori)
            test_images, test_crops, xs,ys = zip(*test_images_crop)

            reconstruction_vals, recon_ori_vals, bn1_val,bn2_val,bn3_val,bn4_val,bn5_val,bn6_val,debn4_val, debn3_val, debn2_val, debn1_val, loss_G_val, loss_D_val = sess.run(
                    [reconstruction, reconstruction_ori, bn1,bn2,bn3,bn4,bn5,bn6,debn4, debn3, debn2, debn1, loss_G, loss_D],
                    feed_dict={
                        images_tf: test_images,
                        images_hiding: test_crops,
                        is_train: False
                        })

            # Generate result every 1000 iterations
            if iters % 100 == 0:
                ii = 0
                for rec_val, img,x,y in zip(reconstruction_vals, test_images, xs, ys):
                    rec_hid = (255. * (rec_val+1)/2.).astype(int)
                    rec_con = (255. * (img+1)/2.).astype(int)

                    rec_con[y:y+64, x:x+64] = rec_hid
                    cv2.imwrite( os.path.join(result_path, 'img_'+str(ii)+'.'+str(int(iters/100))+'.jpg'), rec_con)
                    ii += 1

                if iters == 0:
                    for test_image in test_images_ori:
                        test_image = (255. * (test_image+1)/2.).astype(int)
                        test_image[32:32+64,32:32+64] = 0
                        cv2.imwrite( os.path.join(result_path, 'img_'+str(ii)+'.ori.jpg'), test_image)

            print( "========================================================================")
            print (bn1_val.max(), bn1_val.min())
            print (bn2_val.max(), bn2_val.min())
            print (bn3_val.max(), bn3_val.min())
            print (bn4_val.max(), bn4_val.min())
            print (bn5_val.max(), bn5_val.min())
            print (bn6_val.max(), bn6_val.min())
            print (debn4_val.max(), debn4_val.min())
            print (debn3_val.max(), debn3_val.min())
            print (debn2_val.max(), debn2_val.min())
            print (debn1_val.max(), debn1_val.min())
            print (recon_ori_vals.max(), recon_ori_vals.min())
            print (reconstruction_vals.max(), reconstruction_vals.min())
            print (loss_G_val, loss_D_val)
            print ("=========================================================================")

            if np.isnan(reconstruction_vals.min() ) or np.isnan(reconstruction_vals.max()):
                print ("NaN detected!!")
                ipdb.set_trace()

        # Generative Part is updated every iteration
        _, loss_G_val, adv_pos_val, adv_neg_val, loss_recon_val, loss_adv_G_val, reconstruction_vals, recon_ori_vals, bn1_val,bn2_val,bn3_val,bn4_val,bn5_val,bn6_val,debn4_val, debn3_val, debn2_val, debn1_val = sess.run([train_op_G, loss_G, adversarial_pos, adversarial_neg, loss_recon, loss_adv_G, reconstruction, reconstruction_ori, bn1,bn2,bn3,bn4,bn5,bn6,debn4, debn3, debn2, debn1],feed_dict={images_tf: images,images_hiding: crops,learning_rate: learning_rate_val,is_train: True})

        _, loss_D_val, adv_pos_val, adv_neg_val = sess.run([train_op_D, loss_D, adversarial_pos, adversarial_neg],feed_dict={images_tf: images,images_hiding: crops,learning_rate: learning_rate_val/10.,is_train: True})

        print ("Iter:", iters, "Gen Loss:", loss_G_val, "Recon Loss:", loss_recon_val, "Gen ADV Loss:", loss_adv_G_val,  "Dis Loss:", loss_D_val, "||||", adv_pos_val.mean(), adv_neg_val.min(), adv_neg_val.max())

        iters += 1


    saver.save(sess, model_path + 'model', global_step=epoch)
    learning_rate_val *= 0.99


# Training Dogs dataset
We use the following hyperparameters as discussed in the paper:
- epochs = 30
- learning_rate_ = 0.002
- weight_decay_rate =  0.00001
- momentum = 0.9
- batch_size = 100
- lambda_recon = 0.999
- lambda_adv = 0.001
- overlap_size = 7
- hiding_size = 64

The dataset used here is Dogs dataset **this is our contribution to the project** . 
The dataset consists of 
- Training images =4000 images
- Test images= 100 images

In [5]:
tf.reset_default_graph() 
n_epochs = 30
learning_rate_val = 0.002
weight_decay_rate =  0.00001
momentum = 0.9
batch_size = 100
lambda_recon = 0.999
lambda_adv = 0.001

overlap_size = 7
hiding_size = 64

trainset_path = '../data/dogs_trainset.pickle'
testset_path  = '../data/dogs_testset.pickle'
dataset_path = 'C:/Users/91709/Desktop/CV/dogs'
model_path = 'C:/Users/91709/Desktop/CV/models/dogs/'
result_path= '../results/dogs/'
pretrained_model_path = None

if not os.path.exists(model_path):
    os.makedirs( model_path )

if not os.path.exists(result_path):
    os.makedirs( result_path )

if not os.path.exists( trainset_path ) or not os.path.exists( testset_path ):

    trainset_dir = os.path.join( dataset_path, 'dogs_train' )
    testset_dir = os.path.join( dataset_path, 'dogs_eval' )

    trainset = pd.DataFrame({'image_path': map(lambda x: os.path.join( trainset_dir, x ), os.listdir(trainset_dir))})
    testset = pd.DataFrame({'image_path': map(lambda x: os.path.join( testset_dir, x ), os.listdir(testset_dir))})

    trainset.to_pickle( trainset_path )
    testset.to_pickle( testset_path )
else:
    trainset = pd.read_pickle( trainset_path )
    testset = pd.read_pickle( testset_path )

testset.index = range(len(testset))
testset = testset.loc[np.random.permutation(len(testset))]
is_train = tf.placeholder( tf.bool )

learning_rate = tf.placeholder( tf.float32, [])
images_tf = tf.placeholder( tf.float32, [batch_size, 128, 128, 3], name="images")

labels_D = tf.concat([tf.ones([batch_size]), tf.zeros([batch_size])],0)
labels_G = tf.ones([batch_size])
images_hiding = tf.placeholder( tf.float32, [batch_size, hiding_size, hiding_size, 3], name='images_hiding')

model = Model()

bn1, bn2, bn3, bn4, bn5, bn6, debn4, debn3, debn2, debn1, reconstruction_ori, reconstruction = model.build_reconstruction(images_tf, is_train)
adversarial_pos = model.build_adversarial(images_hiding, is_train)
adversarial_neg = model.build_adversarial(reconstruction, is_train, reuse=True)
adversarial_all = tf.concat([adversarial_pos, adversarial_neg],0)

# Applying bigger loss for overlapping region
mask_recon = tf.pad(tf.ones([hiding_size - 2*overlap_size, hiding_size - 2*overlap_size]), [[overlap_size,overlap_size], [overlap_size,overlap_size]])
mask_recon = tf.reshape(mask_recon, [hiding_size, hiding_size, 1])
mask_recon = tf.concat([mask_recon]*3,2)
mask_overlap = 1 - mask_recon

loss_recon_ori = tf.square( images_hiding - reconstruction )
loss_recon_center = tf.reduce_mean(tf.sqrt( 1e-5 + tf.reduce_sum(loss_recon_ori * mask_recon, [1,2,3])))  # Loss for non-overlapping region
loss_recon_overlap = tf.reduce_mean(tf.sqrt( 1e-5 + tf.reduce_sum(loss_recon_ori * mask_overlap, [1,2,3]))) * 10. # Loss for overlapping region
loss_recon = loss_recon_center + loss_recon_overlap

loss_adv_D = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(logits=adversarial_all, labels=labels_D))
loss_adv_G = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(logits=adversarial_neg, labels=labels_G))

loss_G = loss_adv_G * lambda_adv + loss_recon * lambda_recon
loss_D = loss_adv_D # * lambda_adv

var_G = list(filter( lambda x: x.name.startswith('GEN'), tf.trainable_variables()))
var_D = list(filter( lambda x: x.name.startswith('DIS'), tf.trainable_variables()))

W_G = filter(lambda x: x.name.endswith('W:0'), var_G)
W_D = filter(lambda x: x.name.endswith('W:0'), var_D)

loss_G += weight_decay_rate * tf.reduce_mean(tf.stack( list(map(lambda x: tf.nn.l2_loss(x), W_G))))
loss_D += weight_decay_rate * tf.reduce_mean(tf.stack( list(map(lambda x: tf.nn.l2_loss(x), W_D))))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

optimizer_G = tf.train.AdamOptimizer( learning_rate=learning_rate, beta1=0.5 )
grads_vars_G = optimizer_G.compute_gradients( loss_G, var_list=var_G )
#grads_vars_G = map(lambda gv: [tf.clip_by_value(gv[0], -10., 10.), gv[1]], grads_vars_G)
train_op_G = optimizer_G.apply_gradients( grads_vars_G )

optimizer_D = tf.train.AdamOptimizer( learning_rate=learning_rate, beta1=0.5 )
grads_vars_D = optimizer_D.compute_gradients( loss_D, var_list=var_D )
#grads_vars_D = map(lambda gv: [tf.clip_by_value(gv[0], -10., 10.), gv[1]], grads_vars_D)
train_op_D = optimizer_D.apply_gradients( grads_vars_D )

saver = tf.train.Saver(max_to_keep=100)

tf.initialize_all_variables().run()

if pretrained_model_path is not None and os.path.exists( pretrained_model_path ):
    saver.restore( sess, pretrained_model_path )

iters = 0

loss_D_val = 0.
loss_G_val = 0.

for epoch in range(n_epochs):
    trainset.index = range(len(trainset))
    trainset = trainset.loc[np.random.permutation(len(trainset))]

    for start,end in zip(
            range(0, len(trainset), batch_size),
            range(batch_size, len(trainset), batch_size)):

        image_paths = trainset[start:end]['image_path'].values
        images_ori = list(map(lambda x: load_image( x ), image_paths))

        if iters % 2 == 0:
            images_ori = list(map(lambda img: img[:,::-1,:], images_ori))

        is_none = np.sum(list(map(lambda x: x is None, images_ori)))
        if is_none > 0: continue

        images_crops = list(map(lambda x: crop_random(x, x=32, y=32), images_ori))
        images, crops,_,_ = zip(*images_crops)

        # Printing activations every 100 iterations
        if iters % 100 == 0:
            test_image_paths = testset[:batch_size]['image_path'].values
            test_images_ori = map(lambda x: load_image(x), test_image_paths)

            test_images_crop = map(lambda x: crop_random(x, x=32, y=32), test_images_ori)
            test_images, test_crops, xs,ys = zip(*test_images_crop)

            reconstruction_vals, recon_ori_vals, bn1_val,bn2_val,bn3_val,bn4_val,bn5_val,bn6_val,debn4_val, debn3_val, debn2_val, debn1_val, loss_G_val, loss_D_val = sess.run(
                    [reconstruction, reconstruction_ori, bn1,bn2,bn3,bn4,bn5,bn6,debn4, debn3, debn2, debn1, loss_G, loss_D],
                    feed_dict={
                        images_tf: test_images,
                        images_hiding: test_crops,
                        is_train: False
                        })

            # Generate result every 1000 iterations
            if iters % 100 == 0:
                ii = 0
                for rec_val, img,x,y in zip(reconstruction_vals, test_images, xs, ys):
                    rec_hid = (255. * (rec_val+1)/2.).astype(int)
                    rec_con = (255. * (img+1)/2.).astype(int)

                    rec_con[y:y+64, x:x+64] = rec_hid
                    cv2.imwrite( os.path.join(result_path, 'img_'+str(ii)+'.'+str(int(iters/100))+'.jpg'), rec_con)
                    ii += 1

                if iters == 0:
                    for test_image in test_images_ori:
                        test_image = (255. * (test_image+1)/2.).astype(int)
                        test_image[32:32+64,32:32+64] = 0
                        cv2.imwrite( os.path.join(result_path, 'img_'+str(ii)+'.ori.jpg'), test_image)

            print( "========================================================================")
            print (bn1_val.max(), bn1_val.min())
            print (bn2_val.max(), bn2_val.min())
            print (bn3_val.max(), bn3_val.min())
            print (bn4_val.max(), bn4_val.min())
            print (bn5_val.max(), bn5_val.min())
            print (bn6_val.max(), bn6_val.min())
            print (debn4_val.max(), debn4_val.min())
            print (debn3_val.max(), debn3_val.min())
            print (debn2_val.max(), debn2_val.min())
            print (debn1_val.max(), debn1_val.min())
            print (recon_ori_vals.max(), recon_ori_vals.min())
            print (reconstruction_vals.max(), reconstruction_vals.min())
            print (loss_G_val, loss_D_val)
            print ("=========================================================================")

            if np.isnan(reconstruction_vals.min() ) or np.isnan(reconstruction_vals.max()):
                print ("NaN detected!!")
                ipdb.set_trace()

        # Generative Part is updated every iteration
        _, loss_G_val, adv_pos_val, adv_neg_val, loss_recon_val, loss_adv_G_val, reconstruction_vals, recon_ori_vals, bn1_val,bn2_val,bn3_val,bn4_val,bn5_val,bn6_val,debn4_val, debn3_val, debn2_val, debn1_val = sess.run([train_op_G, loss_G, adversarial_pos, adversarial_neg, loss_recon, loss_adv_G, reconstruction, reconstruction_ori, bn1,bn2,bn3,bn4,bn5,bn6,debn4, debn3, debn2, debn1],feed_dict={images_tf: images,images_hiding: crops,learning_rate: learning_rate_val,is_train: True})

        _, loss_D_val, adv_pos_val, adv_neg_val = sess.run([train_op_D, loss_D, adversarial_pos, adversarial_neg],feed_dict={images_tf: images,images_hiding: crops,learning_rate: learning_rate_val/10.,is_train: True})

        print ("Iter:", iters, "Gen Loss:", loss_G_val, "Recon Loss:", loss_recon_val, "Gen ADV Loss:", loss_adv_G_val,  "Dis Loss:", loss_D_val, "||||", adv_pos_val.mean(), adv_neg_val.min(), adv_neg_val.max())

        iters += 1


    saver.save(sess, model_path + 'model', global_step=epoch)
    learning_rate_val *= 0.99

Instructions for updating:
Use `tf.global_variables_initializer` instead.
967.2388 -101.389755
1000000.0 -100000.0
1000000.0 -100000.0
1000000.0 -100000.0
1000000.0 -100000.0
1000000.0 -100000.0
1000000.0 0.0
1000000.0 0.0
1000000.0 0.0
1000000.0 0.0
214367.81 -219919.52
1.0 -1.0
1129.5441 144445.73
Iter: 0 Gen Loss: 388.31412 Recon Loss: 388.70126 Gen ADV Loss: 0.767843 Dis Loss: 0.7092741 |||| -0.005881141 -0.03927228 0.4927073
Iter: 1 Gen Loss: 385.50986 Recon Loss: 385.89267 Gen ADV Loss: 2.129755 Dis Loss: 0.3468411 |||| 0.8120401 -3.1758537 0.114046305
Iter: 2 Gen Loss: 382.5323 Recon Loss: 382.91046 Gen ADV Loss: 3.6288002 Dis Loss: 0.24373095 |||| 2.2359152 -3.2161639 1.794172
Iter: 3 Gen Loss: 376.28452 Recon Loss: 376.6563 Gen ADV Loss: 3.6364744 Dis Loss: 0.07217056 |||| 2.4522636 -7.432464 -2.5514746
Iter: 4 Gen Loss: 372.54617 Recon Loss: 372.9133 Gen ADV Loss: 4.444803 Dis Loss: 0.037845608 |||| 3.5716393 -6.964196 -2.4466681
Iter: 5 Gen Loss: 383.0651 Recon Loss: 383.442

Iter: 58 Gen Loss: 231.3376 Recon Loss: 231.55717 Gen ADV Loss: 6.8885193 Dis Loss: 0.0028698952 |||| 6.088009 -10.145315 -4.7126365
Iter: 59 Gen Loss: 237.15027 Recon Loss: 237.37483 Gen ADV Loss: 7.6493263 Dis Loss: 0.0020329456 |||| 6.4268866 -10.3845415 -4.920836
Iter: 60 Gen Loss: 240.80635 Recon Loss: 241.03479 Gen ADV Loss: 7.3560305 Dis Loss: 0.0023718965 |||| 6.4293103 -9.210043 -4.714858
Iter: 61 Gen Loss: 239.50038 Recon Loss: 239.72723 Gen ADV Loss: 7.5782056 Dis Loss: 0.002927513 |||| 6.8648524 -8.275056 -3.9380078
Iter: 62 Gen Loss: 236.87624 Recon Loss: 237.1016 Gen ADV Loss: 6.4088287 Dis Loss: 0.0021456766 |||| 6.3557835 -10.178634 -4.639629
Iter: 63 Gen Loss: 232.23035 Recon Loss: 232.44972 Gen ADV Loss: 7.690402 Dis Loss: 0.0030913712 |||| 6.1828504 -11.164337 -4.610378
Iter: 64 Gen Loss: 243.0254 Recon Loss: 243.2547 Gen ADV Loss: 8.503928 Dis Loss: 0.003801695 |||| 6.457927 -7.021912 -3.867027
Iter: 65 Gen Loss: 235.09296 Recon Loss: 235.31677 Gen ADV Loss: 6.03060

Iter: 117 Gen Loss: 205.87094 Recon Loss: 206.05988 Gen ADV Loss: 9.114497 Dis Loss: 0.0005890623 |||| 7.464624 -11.842564 -7.080006
Iter: 118 Gen Loss: 218.68967 Recon Loss: 218.89088 Gen ADV Loss: 9.591427 Dis Loss: 0.0006527532 |||| 7.4559984 -11.674257 -7.0352616
Iter: 119 Gen Loss: 216.65614 Recon Loss: 216.85555 Gen ADV Loss: 9.332302 Dis Loss: 0.0004897434 |||| 7.9609284 -11.014358 -5.7336264
Iter: 120 Gen Loss: 214.40607 Recon Loss: 214.60394 Gen ADV Loss: 8.558464 Dis Loss: 0.0010558887 |||| 7.876215 -10.536978 -4.4567747
Iter: 121 Gen Loss: 214.2661 Recon Loss: 214.46521 Gen ADV Loss: 7.1514525 Dis Loss: 0.00072229316 |||| 8.139944 -9.262059 -4.4953103
Iter: 122 Gen Loss: 215.53552 Recon Loss: 215.73509 Gen ADV Loss: 7.9217467 Dis Loss: 0.00046779957 |||| 8.065851 -11.218848 -5.845026
Iter: 123 Gen Loss: 208.5002 Recon Loss: 208.69171 Gen ADV Loss: 8.911175 Dis Loss: 0.0006399682 |||| 7.5455766 -12.122566 -7.111809
Iter: 124 Gen Loss: 209.4657 Recon Loss: 209.6577 Gen ADV Los

Iter: 179 Gen Loss: 204.77086 Recon Loss: 204.9554 Gen ADV Loss: 9.760978 Dis Loss: 0.00029775003 |||| 8.62724 -12.325468 -7.390116
Iter: 180 Gen Loss: 198.96033 Recon Loss: 199.13956 Gen ADV Loss: 9.227502 Dis Loss: 0.000449586 |||| 8.357331 -10.541562 -6.4331055
Iter: 181 Gen Loss: 192.52048 Recon Loss: 192.69424 Gen ADV Loss: 8.174565 Dis Loss: 0.00048338718 |||| 8.368251 -10.988181 -6.763397
Iter: 182 Gen Loss: 196.49504 Recon Loss: 196.67212 Gen ADV Loss: 8.812735 Dis Loss: 0.00045095955 |||| 8.658133 -10.115421 -5.9774785
Iter: 183 Gen Loss: 222.9277 Recon Loss: 223.13148 Gen ADV Loss: 8.524627 Dis Loss: 0.0004620977 |||| 8.079342 -11.8745985 -7.2649617
Iter: 184 Gen Loss: 195.474 Recon Loss: 195.64902 Gen ADV Loss: 9.772094 Dis Loss: 0.0003362908 |||| 8.262605 -12.452665 -7.01757
Iter: 185 Gen Loss: 223.60587 Recon Loss: 223.80864 Gen ADV Loss: 10.112611 Dis Loss: 0.0003174986 |||| 8.826227 -11.472285 -6.7170305
Iter: 186 Gen Loss: 203.34349 Recon Loss: 203.52724 Gen ADV Loss: 8

Iter: 238 Gen Loss: 202.05396 Recon Loss: 202.23294 Gen ADV Loss: 10.097119 Dis Loss: 0.00031124154 |||| 9.050695 -12.026803 -6.5879426
Iter: 239 Gen Loss: 210.11177 Recon Loss: 210.29987 Gen ADV Loss: 9.033227 Dis Loss: 0.00043339265 |||| 8.76172 -10.564774 -5.8380547
Iter: 240 Gen Loss: 193.23882 Recon Loss: 193.41019 Gen ADV Loss: 8.8154335 Dis Loss: 0.00028570692 |||| 9.137256 -10.846731 -5.6453667
Iter: 241 Gen Loss: 190.37859 Recon Loss: 190.54724 Gen ADV Loss: 8.64794 Dis Loss: 0.00028278714 |||| 8.941408 -11.922947 -6.517758
Iter: 242 Gen Loss: 192.01932 Recon Loss: 192.18886 Gen ADV Loss: 9.341419 Dis Loss: 0.00032751993 |||| 8.718692 -12.286104 -6.8033476
Iter: 243 Gen Loss: 193.42343 Recon Loss: 193.59427 Gen ADV Loss: 9.414975 Dis Loss: 0.00026637825 |||| 8.459139 -12.85082 -8.128083
Iter: 244 Gen Loss: 183.94975 Recon Loss: 184.11029 Gen ADV Loss: 10.18269 Dis Loss: 0.00037757383 |||| 8.5318165 -12.477951 -7.308596
Iter: 245 Gen Loss: 198.53053 Recon Loss: 198.70607 Gen AD

...

Iter: 899 Gen Loss: 150.54066 Recon Loss: 150.64056 Gen ADV Loss: 9.710702 Dis Loss: 0.0002869934 |||| 11.019917 -13.624638 -5.044737
8.991085 -0.9567404
7.226412 -0.67630315
7.736996 -0.84961814
7.662523 -0.67337483
5.8641734 -0.56935954
5.7251315 -0.5034167
7.1842427 0.0
10.408506 0.0
14.520626 0.0
28.040615 0.0
2.34277 -2.1880639
0.9817133 -0.9751644
141.18755 0.0002794982
Iter: 900 Gen Loss: 155.43051 Recon Loss: 155.53577 Gen ADV Loss: 9.171854 Dis Loss: 0.00028315344 |||| 10.940809 -12.805637 -5.2566442
Iter: 901 Gen Loss: 157.98792 Recon Loss: 158.09521 Gen ADV Loss: 9.596785 Dis Loss: 0.00025975175 |||| 11.263089 -12.803368 -6.885309
Iter: 902 Gen Loss: 156.03264 Recon Loss: 156.13754 Gen ADV Loss: 10.005672 Dis Loss: 0.0004640459 |||| 11.120091 -13.531548 -4.209031
Iter: 903 Gen Loss: 150.97734 Recon Loss: 151.07869 Gen ADV Loss: 8.488302 Dis Loss: 0.00042098403 |||| 10.807095 -11.582016 -5.1187215
Iter: 904 Gen Loss: 153.15033 Recon Loss: 153.25351 Gen ADV Loss: 8.814946 Dis 

Iter: 957 Gen Loss: 153.9444 Recon Loss: 154.04538 Gen ADV Loss: 9.857883 Dis Loss: 0.00041471992 |||| 10.403604 -15.252094 -6.3757715
Iter: 958 Gen Loss: 162.33609 Recon Loss: 162.4445 Gen ADV Loss: 10.769714 Dis Loss: 0.00018719905 |||| 11.5255995 -15.185993 -6.888441
Iter: 959 Gen Loss: 167.30327 Recon Loss: 167.41765 Gen ADV Loss: 9.772399 Dis Loss: 0.00035792452 |||| 10.147052 -14.788721 -6.748448
Iter: 960 Gen Loss: 145.63872 Recon Loss: 145.7309 Gen ADV Loss: 10.212786 Dis Loss: 0.00021246668 |||| 10.82401 -13.669999 -6.6329837
Iter: 961 Gen Loss: 142.80685 Recon Loss: 142.8956 Gen ADV Loss: 10.767474 Dis Loss: 0.00020971155 |||| 11.269585 -14.893591 -6.5168595
Iter: 962 Gen Loss: 148.38937 Recon Loss: 148.4842 Gen ADV Loss: 10.236274 Dis Loss: 0.00016857599 |||| 11.08439 -14.432352 -7.0743256
Iter: 963 Gen Loss: 146.51096 Recon Loss: 146.60391 Gen ADV Loss: 10.2504 Dis Loss: 0.00024347095 |||| 10.772383 -12.974642 -6.219415
Iter: 964 Gen Loss: 149.51566 Recon Loss: 149.61267 Ge

Iter: 1015 Gen Loss: 141.02248 Recon Loss: 141.10896 Gen ADV Loss: 9.412934 Dis Loss: 0.0002854096 |||| 10.662675 -13.876015 -7.032523
Iter: 1016 Gen Loss: 147.15804 Recon Loss: 147.24982 Gen ADV Loss: 10.186514 Dis Loss: 0.00020392705 |||| 10.832191 -14.72034 -6.962552
Iter: 1017 Gen Loss: 159.16946 Recon Loss: 159.27345 Gen ADV Loss: 9.947832 Dis Loss: 0.00018413438 |||| 11.9315405 -13.646805 -6.457824
Iter: 1018 Gen Loss: 150.56813 Recon Loss: 150.66382 Gen ADV Loss: 9.6090355 Dis Loss: 0.0004276226 |||| 10.631129 -12.678532 -4.8552556
Iter: 1019 Gen Loss: 150.47629 Recon Loss: 150.5718 Gen ADV Loss: 9.641269 Dis Loss: 0.00023445532 |||| 12.814283 -12.877184 -5.782468
Iter: 1020 Gen Loss: 161.78487 Recon Loss: 161.89308 Gen ADV Loss: 8.201838 Dis Loss: 0.00030487607 |||| 10.944788 -13.271041 -6.432478
Iter: 1021 Gen Loss: 141.35364 Recon Loss: 141.43953 Gen ADV Loss: 10.020763 Dis Loss: 0.0006719288 |||| 9.467629 -14.708489 -7.0054355
Iter: 1022 Gen Loss: 155.67358 Recon Loss: 155.7

Iter: 1076 Gen Loss: 149.05096 Recon Loss: 149.14218 Gen ADV Loss: 10.527899 Dis Loss: 0.00014215997 |||| 12.675734 -13.756787 -6.7092104
Iter: 1077 Gen Loss: 150.83327 Recon Loss: 150.92755 Gen ADV Loss: 9.211954 Dis Loss: 0.00014923495 |||| 12.691178 -14.403037 -6.9593396
Iter: 1078 Gen Loss: 145.49356 Recon Loss: 145.58107 Gen ADV Loss: 10.600328 Dis Loss: 0.00012865734 |||| 11.993404 -14.223935 -7.8025646
Iter: 1079 Gen Loss: 144.22232 Recon Loss: 144.30855 Gen ADV Loss: 10.568204 Dis Loss: 0.00016886383 |||| 11.891836 -14.429914 -6.3815813
Iter: 1080 Gen Loss: 154.16902 Recon Loss: 154.26556 Gen ADV Loss: 10.190439 Dis Loss: 0.00020734513 |||| 11.64482 -13.639249 -5.9123335
Iter: 1081 Gen Loss: 147.41193 Recon Loss: 147.50171 Gen ADV Loss: 10.103124 Dis Loss: 0.00017521065 |||| 10.980573 -14.7405815 -7.1247077
Iter: 1082 Gen Loss: 151.7425 Recon Loss: 151.83582 Gen ADV Loss: 10.870029 Dis Loss: 0.00013257553 |||| 11.772996 -15.074013 -8.171711
Iter: 1083 Gen Loss: 153.62584 Recon 

Iter: 1134 Gen Loss: 143.44093 Recon Loss: 143.52469 Gen ADV Loss: 10.360974 Dis Loss: 0.00022282405 |||| 11.24608 -12.9950485 -6.2548223
Iter: 1135 Gen Loss: 145.36053 Recon Loss: 145.44647 Gen ADV Loss: 10.072977 Dis Loss: 0.00016387044 |||| 12.120364 -14.969773 -5.9005775
Iter: 1136 Gen Loss: 145.63454 Recon Loss: 145.72018 Gen ADV Loss: 10.602364 Dis Loss: 0.0001954785 |||| 12.39126 -14.390532 -5.6252093
Iter: 1137 Gen Loss: 145.75175 Recon Loss: 145.83765 Gen ADV Loss: 10.449344 Dis Loss: 0.00016957315 |||| 12.455471 -14.038645 -6.3406715
Iter: 1138 Gen Loss: 143.24841 Recon Loss: 143.33157 Gen ADV Loss: 10.615048 Dis Loss: 0.00022093298 |||| 11.921981 -15.723904 -5.9624095
Iter: 1139 Gen Loss: 145.9087 Recon Loss: 145.99487 Gen ADV Loss: 10.247671 Dis Loss: 0.00016512148 |||| 13.074041 -14.217605 -5.4591336
Iter: 1140 Gen Loss: 143.99823 Recon Loss: 144.08217 Gen ADV Loss: 10.5295925 Dis Loss: 0.00017164637 |||| 11.751737 -14.797561 -7.4721646
Iter: 1141 Gen Loss: 148.34319 Recon